In [1]:
import numpy as np 
import open3d as o3d 
import matplotlib.pyplot as plt
from pathlib import Path

from numpy.ma.core import min_filler

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
data_name = Path('../data/tutorial/appartment_cloud.ply')
pcd = o3d.io.read_point_cloud(str(data_name))

In [3]:
pcd_center = pcd.get_center()
print(pcd_center)
pcd.translate(-pcd_center)

[-6.60738023  2.74615613 -0.24302214]


PointCloud with 1430779 points.

In [4]:
o3d.core.cuda.is_available()

False

In [5]:
print(pcd.get_center())

[ 1.53784121e-13  5.12532177e-14 -3.44614299e-13]


### Statistical outlier filter

In [14]:
nn = 16 # nearest neighbours 
std_multiplier = 10 # it is is above that, it is considered outlier

filtered_pcd = pcd.remove_statistical_outlier(nn, std_multiplier)

In [15]:
outliers = pcd.select_by_index(filtered_pcd[1], invert=True)

In [17]:
outliers.paint_uniform_color([0,1,0])
pcd_ww_outliers = filtered_pcd[0]

o3d.visualization.draw_geometries([pcd_ww_outliers, outliers])

### Voxel downsampling

In [21]:
voxel_size = 0.01 # 10 cm 

pcd_downsampled = pcd_ww_outliers.voxel_down_sample(voxel_size)
o3d.visualization.draw_geometries([pcd_downsampled])

### Estimating normals

In [23]:
nn_distance = np.mean(pcd.compute_nearest_neighbor_distance())
nn_distance

0.0051852883612620694

In [24]:
radius_normals = nn_distance * 4 
pcd_downsampled.estimate_normals(search_param = o3d.geometry.KDTreeSearchParamHybrid(radius = radius_normals, max_nn = nn), fast_normal_computation=True)

Ctrl + c

{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 2.0468802265837804, 2.0653438714155197, 1.2155221442301669 ],
			"boundingbox_min" : [ -2.0644531067495531, -2.01065612858448, -1.2809778557698333 ],
			"field_of_view" : 60.0,
			"front" : [ 0.98494375440722048, 0.16774148106653808, -0.041816219147759882 ],
			"lookat" : [ -0.0041572339440813831, -0.10649480480045723, -0.46057117739954029 ],
			"up" : [ 0.043517653141294972, -0.0064785057448386559, 0.99903165256581739 ],
			"zoom" : 0.27999999999999969
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}

In [26]:
pcd_downsampled.paint_uniform_color([0.6, 0.6, 0.6])
o3d.visualization.draw_geometries([pcd_downsampled, outliers])

In [27]:
front = [ 0.98494375440722048, 0.16774148106653808, -0.041816219147759882 ]
lookat = [ -0.0041572339440813831, -0.10649480480045723, -0.46057117739954029 ]
up = [ 0.043517653141294972, -0.0064785057448386559, 0.99903165256581739 ]
zoom = 0.27999999999999969

In [28]:
pcd = pcd_downsampled
o3d.visualization.draw_geometries([pcd], zoom=zoom, front=front, lookat=lookat, up=up)

### RANSAC Planar Segmentation

In [30]:
pt_to_plane_dist = 0.02 # how far it should be maximally to be considered inlier 

plane_model, inliers = pcd.segment_plane(distance_threshold=pt_to_plane_dist, ransac_n=3, num_iterations=1000)
[a, b, c, d] = plane_model

inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)
inlier_cloud.paint_uniform_color([0,1,0])
outlier_cloud.paint_uniform_color([0.6, 0.6, 0.6])

o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud], zoom=zoom, front=front, lookat=lookat, up=up)

### Multi-Order RANSAC

In [31]:
max_plane_idx = 6
pt_to_plane_dist = 0.02 

segment_models = {}
segments = {}
rest = pcd

for i in range(max_plane_idx):
    colors = plt.get_cmap('tab20')(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=pt_to_plane_dist, ransac_n=3, num_iterations=1000)   
    segments[i] = rest.select_by_index(inliers)
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print(f"pass {i} / {max_plane_idx} done.")

pass 0 / 6 done.
pass 1 / 6 done.
pass 2 / 6 done.
pass 3 / 6 done.
pass 4 / 6 done.
pass 5 / 6 done.


In [32]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)] + [rest], 
                                  zoom=zoom, front=front, lookat=lookat, up=up)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


### Euclidean Clustering Refine
Use remaining points to cluster them. 

In [33]:
labels = np.array(rest.cluster_dbscan(eps=0.05, min_points=5))
max_label = labels.max()
print(f"point cloud has {max_label + 1} labels")

point cloud has 8 labels


In [34]:
colors = plt.get_cmap('tab20')(labels / min(max_label, 1))
colors[labels < 0] = 0
rest.colors = o3d.utility.Vector3dVector(colors[:, :3])

In [36]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)] + [rest], 
                                  zoom=zoom, front=front, lookat=lookat, up=up)